# Building a Linear Regression Model

In [0]:
import tensorflow as tf
import numpy as np

Reset Default graph - Needed only for Jupyter notebook

In [0]:
tf.reset_default_graph()

# Step 1 - Collect Data
Use Boston housing prices data available in tensorflow

In [0]:
from tensorflow.contrib.learn import datasets
boston = datasets.load_dataset('boston')

#Step 1a - Extract input features and Target

In [0]:
#Input features
features = np.array(boston.data)

#Actual output
prices = np.array(boston.target)

In [0]:
print('Input features shape: ', features.shape)
print('Actual Prices data shape: ', prices.shape)

In [0]:
#Reshape prices to make it work
prices = np.reshape(prices,[-1,1])

How many examples do we have?

# Building the Graph

Define input data placeholders as actual data will be provided at run time

In [0]:
#Input features placeholder
x = tf.placeholder(shape=[None,features.shape[1]],dtype=tf.float32, name='x-input')

#Actual Price placeholder
y_ = tf.placeholder(shape=[None,prices.shape[1]],dtype=tf.float32, name='y-input')

In [0]:
#Normalize input features data
x_n = tf.nn.l2_normalize(x,1)

Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[features.shape[1],1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

**Prediction**
Make sure to use normalized data -> x_n

In [0]:
#y = x_n*W + b
y = tf.add(tf.matmul(x_n,W),b,name='output')

Loss (Cost) Function

In [0]:
#Mean Suared error
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

GradientDescent Optimizer to minimize Loss

In [0]:
learn_rate = 0.03
train_op = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)

Logging information which can be used with TensorBoard

In [0]:
#Define a summary scalar object to capture loss
training_loss = tf.summary.scalar('train_loss',loss)

#Define the logs directory to store the logs, you can change the value
log_dir = '/tmp/boston/v1' 

#Initialize Writer for Logging
writer = tf.summary.FileWriter(log_dir, graph=tf.get_default_graph())

#Create saver to save the graph
saver = tf.train.Saver()

# Executing the Graph

In [0]:
#Lets start graph Execution
with tf.Session() as sess:
  
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())
    
    #how many times data need to be shown to model
    training_epochs = 1000     
    
    
    for epoch in range(training_epochs):
        
        #Calculate train_op and loss
        _, train_loss, loss_log = sess.run([train_op,  #Gradient Descent node
                                            loss,      #loss node for printing
                                            training_loss], #stored in log file
                                           feed_dict={x:features, y_:prices})
        
        #Write the logs
        writer.add_summary(loss_log, epoch)
        
        #Print the loss after every 100 iterations
        if epoch % 100 == 0:
            print ('Training loss at step: ', epoch, ' is ', train_loss)
            
    #Saving the graph
    saver.save(sess, log_dir + '/' + 'boston.ckpt')

How do we view the logs and graph using TensorBoard? 